In [1]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [2]:
import math
import time

import numpy as np

import matplotlib.pyplot as _mpyplot
import matplotlib.gridspec as _mgridspec
import matplotlib.cm as _cm
import matplotlib.colors as _mcolors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

from siriuspy.clientconfigdb import ConfigDBClient
import pyaccel
import mathphys
from pymodels import si
from siriuspy.devices import PowerSupply

In [3]:
clt = ConfigDBClient(config_type='si_orbit')

correctorsx = []
correctorsy = []
for idx in range(1, 5):
    cname = f'SI-09SA:PS-CH-{idx}'
    print(cname)
#     correctorsx.append(PowerSupply(cname))
    cname = f'SI-09SA:PS-CV-{idx}'
    print(cname)
#     correctorsy.append(PowerSupply(cname))

SI-09SA:PS-CH-1
SI-09SA:PS-CV-1
SI-09SA:PS-CH-2
SI-09SA:PS-CV-2
SI-09SA:PS-CH-3
SI-09SA:PS-CV-3
SI-09SA:PS-CH-4
SI-09SA:PS-CV-4


In [4]:
def calc_corr_strength(ang, pos=0):
    """Calc bump for straight section 09SA.
    Inputs:
        ang - bump angle at straight center in [urad].
        pos - bum offset at straight center in [um].
        
    Outputs:
        kicks - correctors strength in [urad].
    
    Geometry:
    >----------------> ebeam direction >---------------->
      C1|C1      C2|C2                     C3|C3      C4|C4
        |---len1---|----len2----|----len2----|---len1---|
    """
    len1 = 0.634
    len2 = 2.53525
    
    #angle bump
    theta = math.atan(len2/len1 * math.tan(ang/1e6)) * 1e6 
    corrs = [-theta, theta + ang, -theta - ang, theta] 
     
    # offset bump 
    theta = math.atan(pos/1e6 / len1) * 1e6 
    corrs[0] += theta 
    corrs[1] -= theta 
    corrs[2] -= theta 
    corrs[3] += theta 
    return np.asarray(corrs)


def create_and_apply_bump_with_id_corrs(angx, angy=0, posx=0, posy=0):
    """Calc reference orbit for bumps for SOFB.
    Inputs:
        angx - Horizontal bump angle at straight center in [urad].
        posx - Horizontal bump offset at straight center in [um].
        angy - Vertical bump angle at straight center in [urad].
        posy - Vertical bump offset at straight center in [um].
    """
    maxkick = 32 * 4.8  # maximum corrector strength
    maxdlt = 10  # maximum delta to apply
    
    kickx0 = np.asarray([c.strength for c in correctorsx])
    kicky0 = np.asarray([c.strength for c in correctorsy])
    
    kickx = calc_corr_strength(angx, posx)
    kicky = calc_corr_strength(angy, posy)
    if np.any(kickx > maxkick) or np.any(kicky > maxkick):
        print('Cannot apply bump. Correctors strength would be exceeded.')
    dkickx = kickx - kickx0
    dkicky = kicky - kicky0
    mcorrx = np.max(np.abs(dkickx))
    mcorry = np.max(np.abs(dkicky))
    mcorr = max(mcorrx, mcorry)
    
    nrsteps = int(mcorr//maxdlt) + 2
    factors = np.linspace(0, 1, nrsteps)[1:]
    for fac in factors:
        for corr, k0, dk in zip(correctorsx, kickx0, dkickx):
            corr.strength = k0 + fac*dk
        for corr, k0, dk in zip(correctorsy, kicky0, dkicky):
            corr.strength = k0 + fac*dk
        time.sleep(1)

        
def create_bump_ref_for_sofb(configname, angx, angy=0, posx=0, posy=0):
    """Calc reference orbit for bumps for SOFB.
    Inputs:
        configname - name of the configuration to be generated.
        angx - Horizontal bump angle at straight center in [urad].
        posx - Horizontal bump offset at straight center in [um].
        angy - Vertical bump angle at straight center in [urad].
        posy - Vertical bump offset at straight center in [um].
    """
    ss_len = 6.1758 / 2  # distance between BPMs.
    bpm_m1 = 8*8 - 1  # SI-09M1:DI-BPM
    bpm_m2 = 8*8      # SI-09M2:DI-BPM
    
    orb = clt.get_config_value('bba_orb')
    
    # angle bump
    dx = ss_len * math.tan(angx/1e6) * 1e6
    orb['x'][bpm_m1] -= dx
    orb['x'][bpm_m2] += dx
    dy = ss_len * math.tan(angy/1e6) * 1e6
    orb['y'][bpm_m1] -= dy
    orb['y'][bpm_m2] += dy
    
    #pos bump
    orb['x'][bpm_m1] += posx
    orb['x'][bpm_m2] += posx
    orb['y'][bpm_m1] += posy
    orb['y'][bpm_m2] += posy
    
    clt.insert_config(configname, orb)

In [7]:
calc_corr_strength(30, )

array([ -88.41876919,  118.41876919, -181.51025182,  151.51025182])